
# Análisis cuantitativo del sistema inmunitario humano (interactivo)
## Efecto de un aumento del 20% en la masa del tejido adiposo

**Autor:** Raúl Mendoza  
**Asignatura:** Bioinformática  

Este cuaderno analiza los datos suplementarios de Ron Milo et al. (PNAS, 2023) para:
1. describir la distribución basal de células inmunitarias por tejido,
2. visualizarla de forma **interactiva**,
3. simular un escenario de **+20% de masa de tejido adiposo**,
4. cuantificar el impacto sobre el total de células y masa inmunitaria,
5. interpretar los resultados y sus limitaciones.

### Pregunta de investigación
> **¿Hasta qué punto el aumento de la masa del tejido adiposo (como modelo simple de obesidad) modifica el reparto cuantitativo de las células inmunitarias en el organismo?**


In [14]:

import os
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from IPython.display import display, HTML

# función segura para mostrar dentro del notebook
def mostrar(fig):
    try:
        fig.show()
    except Exception:
        # fallback: incrustar como HTML dentro del notebook
        html = fig.to_html(include_plotlyjs="cdn")
        display(HTML(html))

SD01 = "pnas.2308511120.sd01.xlsx"
SD02 = "pnas.2308511120.sd02.xlsx"

assert os.path.exists(SD01), "No se encuentra pnas.2308511120.sd01.xlsx"
assert os.path.exists(SD02), "No se encuentra pnas.2308511120.sd02.xlsx"

sd01 = pd.ExcelFile(SD01)
sd02 = pd.ExcelFile(SD02)

tissues = sd01.parse("Tissues")
mass = sd01.parse("Mass")
cell_type_densities = sd02.parse("cell_type_densities")
total_immune_densities = sd02.parse("total_immune_densities")
numbers_by_cell_type = sd02.parse("numbers_by_cell_type")
mass_by_cell_type = sd02.parse("mass_by_cell_type")
totals = sd02.parse("totals")

def clean_df(df: pd.DataFrame) -> pd.DataFrame:
    df = df.copy()
    df = df.loc[:, ~df.columns.astype(str).str.contains("^Unnamed")]
    df.columns = [str(c).strip() for c in df.columns]
    return df

tissues = clean_df(tissues)
mass = clean_df(mass)
cell_type_densities = clean_df(cell_type_densities)
total_immune_densities = clean_df(total_immune_densities)
numbers_by_cell_type = clean_df(numbers_by_cell_type)
mass_by_cell_type = clean_df(mass_by_cell_type)
totals = clean_df(totals)

tissues.head()


,tissue,subtissue,comment,mass,sem,groups,front line organ,for_final_table,child_10y,woman,man
0,Blood,NaN,NaN,5241.0,NaN,blood,NaN,Blood,2616.0,4251.0,5500.0
1,Red marrow,NaN,NaN,1500.0,NaN,marrow,NaN,Bone Marrow,630.0,900.0,1170.0
2,Lymph nodes,NaN,NaN,250.0,NaN,lymph system,NaN,Lymph system,99.0,185.0,225.0
3,Lymph vessels,NaN,NaN,450.0,NaN,connective,NaN,Lymph system,221.0,415.0,505.0
4,Spleen,NaN,NaN,150.0,20.0,lymph system,NaN,Lymph system,80.0,130.0,150.0



## 1. Descripción de los datos

- **Tissues**: masa de cada tejido en gramos (`man` = individuo de referencia).
- **total_immune_densities**: densidad total de células inmunitarias por tejido.
- **cell_type_densities**: densidades específicas de cada tipo celular en cada tejido.
- **numbers_by_cell_type**: número total de cada tipo de célula para el individuo de referencia.
- **mass_by_cell_type**: masa total de cada tipo de célula.

Con esto podemos pasar de *tejido → densidad → número → masa*.


In [15]:

total_cells_ref = numbers_by_cell_type["tot_man"].sum()
total_mass_ref_g = mass_by_cell_type["mass"].sum()

avg_mass_per_cell = (
    mass_by_cell_type[["cell_type", "mass"]]
    .merge(numbers_by_cell_type[["cell_type", "tot_man"]], on="cell_type")
    .assign(g_per_cell=lambda d: d["mass"] / d["tot_man"])
    .set_index("cell_type")["g_per_cell"]
)

print(f"Total de células inmunitarias (referencia): {total_cells_ref:,.0f}")
print(f"Masa inmunitaria total (referencia): {total_mass_ref_g:,.2f} g")


Total de células inmunitarias (referencia): 1,837,950,387,444
Masa inmunitaria total (referencia): 1,224.49 g



**Interpretación:** estos valores son la línea base del individuo de referencia (73 kg). Toda la simulación posterior se comparará contra ellos para ver si el aumento del tejido adiposo cambia de verdad el “peso” del sistema inmunitario o si el efecto es más bien local.


In [16]:

# Figura 1: tejidos más densos inmunológicamente
top_density = (
    total_immune_densities[["tissue", "density"]]
    .sort_values("density", ascending=False)
    .head(15)
)

fig1 = px.bar(
    top_density,
    x="tissue",
    y="density",
    title="Figura 1. Tejidos con mayor densidad de células inmunitarias",
)
fig1.update_layout(xaxis_title="Tejido", yaxis_title="Células/g")
mostrar(fig1)



**Figura 1.** Los tejidos que aparecen arriba son los que concentran más células inmunitarias por gramo. Esto es clave para justificar qué tejidos son “inmunológicamente activos”: no basta con que un órgano sea grande, tiene que estar densamente poblado. Los tejidos linfoides deberían situarse aquí porque su función es precisamente esa.


In [17]:

# Figura 2: treemap de contribución aproximada por tejido
# aproximamos "carga inmune" = masa del tejido * densidad inmune
tissue_load = (
    tissues[["tissue", "man"]]
    .merge(total_immune_densities[["tissue", "density"]], on="tissue", how="inner")
    .assign(immune_load=lambda d: d["man"] * d["density"])
    .sort_values("immune_load", ascending=False)
)

fig2 = px.treemap(
    tissue_load,
    path=["tissue"],
    values="immune_load",
    title="Figura 2. Contribución aproximada de cada tejido a la carga inmunitaria total",
)
mostrar(fig2)



**Figura 2.** A diferencia de la barra, aquí vemos el **peso relativo** de cada tejido en el conjunto del cuerpo. Tejidos muy voluminosos pero menos densos (como sangre o médula ósea) pueden aparecer grandes porque su volumen compensa la menor densidad. Esta figura responde muy bien al enfoque “¿qué tejidos aportan más al sistema inmune en total?”.


In [18]:

# Figura 3: masa del tejido vs densidad (log-log)
df_mass_density = (
    tissues[["tissue", "man"]]
    .merge(total_immune_densities[["tissue", "density"]], on="tissue", how="inner")
    .rename(columns={"man": "tissue_mass_g", "density": "immune_density_cells_g"})
)

fig3 = px.scatter(
    df_mass_density,
    x="tissue_mass_g",
    y="immune_density_cells_g",
    hover_name="tissue",
    title="Figura 3. Masa del tejido vs densidad inmunitaria (log-log)",
)
fig3.update_xaxes(type="log", title="Masa del tejido (g, log)")
fig3.update_yaxes(type="log", title="Densidad inmunitaria (células/g, log)")
mostrar(fig3)



**Figura 3.** Esta nube interactiva muestra que no hay correlación simple entre masa y densidad: hay tejidos pequeños con mucha actividad inmune y tejidos grandes con menos. El usuario puede pasar el ratón por encima para ver el nombre del tejido y así identificar casos atípicos. Esta interacción es útil para la discusión del informe.


In [19]:

# Figura 4: distribución de masa por tipo celular (sunburst)
mass_sorted = mass_by_cell_type.sort_values("mass", ascending=False)
fig4 = px.sunburst(
    mass_sorted,
    path=["cell_type"],
    values="mass",
    title="Figura 4. Distribución de la masa inmunitaria por tipo celular",
)
mostrar(fig4)



**Figura 4.** El sunburst permite ver de un vistazo qué tipos celulares pesan más dentro del sistema inmunitario. Aunque algunos no sean los más numerosos, su masa los hace relevantes (por ejemplo, macrófagos). Esta figura apoya la idea de que para entender el sistema inmune no basta con contar células: también hay que mirar la masa que ocupan.


### Simulación: +20% tejido adiposo

In [20]:

# 2. Simulación: +20% tejido adiposo

# localizar tejido adiposo
adipose_row = df_mass_density[df_mass_density["tissue"].str.contains("Adipose", case=False)]
if adipose_row.empty:
    raise ValueError("No se encontró el tejido adiposo. Revisa el nombre exacto en la hoja 'Tissues'.")

adipose_mass_ref_g = float(adipose_row["tissue_mass_g"].iloc[0])

# densidades específicas
adipose_cell_dens = cell_type_densities[
    cell_type_densities["tissue"].str.contains("Adipose", case=False)
][["cell_type", "density"]].copy()

# números base y nuevos
adipose_numbers_base = adipose_cell_dens.assign(
    number=lambda d: d["density"] * adipose_mass_ref_g
)
adipose_mass_new_g = adipose_mass_ref_g * 1.20
adipose_numbers_new = adipose_cell_dens.assign(
    number=lambda d: d["density"] * adipose_mass_new_g
)

# diferencias
delta_numbers = (
    adipose_numbers_new.set_index("cell_type")["number"]
    - adipose_numbers_base.set_index("cell_type")["number"]
)

delta_cells_total = float(delta_numbers.sum())
delta_mass_total_g = float(
    (delta_numbers * avg_mass_per_cell.reindex(delta_numbers.index)).sum()
)

print(f"Células adicionales por +20% adiposo: {delta_cells_total:,.0f}")
print(f"Masa inmunitaria adicional estimada: {delta_mass_total_g:,.4f} g")


Células adicionales por +20% adiposo: 1,135,272,491
Masa inmunitaria adicional estimada: 2.3801 g



**Interpretación del escenario:** al aumentar la masa del tejido adiposo manteniendo su densidad, todo el contenido inmunitario de ese tejido crece de forma proporcional. Esto imita una obesidad “geométrica” (más volumen, mismo tejido). El resultado esperado es un aumento absoluto de células inmunes, pero tendremos que comparar ese aumento con el total del organismo para saber si realmente es relevante.


In [21]:

# Figura 5: comparación antes/después en tejido adiposo
compare_df = adipose_numbers_base.merge(
    adipose_numbers_new, on="cell_type", suffixes=("_ref", "_new")
)

fig5 = go.Figure(data=[
    go.Bar(name="Escenario basal", x=compare_df["cell_type"], y=compare_df["number_ref"]),
    go.Bar(name="+20% adiposo", x=compare_df["cell_type"], y=compare_df["number_new"])
])
fig5.update_layout(
    barmode="group",
    title="Figura 5. Células inmunes en tejido adiposo: antes vs después",
    xaxis_title="Tipo celular",
    yaxis_title="Células estimadas"
)
mostrar(fig5)



**Figura 5.** Todas las barras del escenario “+20% adiposo” están por encima de las basales porque el modelo es proporcional. La utilidad de esta figura es que permite ver **qué tipos celulares había realmente en el tejido adiposo**: los que tienen barras más grandes son los que más contribuyen al aumento total. Esto se puede comentar en el informe como posible punto de inflamación local.


In [22]:

# Tabla de indicadores clave
kpi_df = pd.DataFrame({
    "Indicador": [
        "Células inmunitarias totales (referencia)",
        "Masa inmunitaria total (referencia)",
        "Células adicionales (+20% adiposo)",
        "Masa inmunitaria adicional (+20% adiposo)",
        "Aumento relativo de masa inmunitaria",
    ],
    "Valor": [
        f"{total_cells_ref:,.0f}",
        f"{total_mass_ref_g:,.2f} g",
        f"{delta_cells_total:,.0f}",
        f"{delta_mass_total_g:,.4f} g",
        f"{(delta_mass_total_g/total_mass_ref_g)*100:.5f} %",
    ]
})
display(kpi_df)


,Indicador,Valor
0,Células inmunitarias totales (referencia),"1,837,950,387,444"
1,Masa inmunitaria total (referencia),"1,224.49 g"
2,Células adicionales (+20% adiposo),"1,135,272,491"
3,Masa inmunitaria adicional (+20% adiposo),2.3801 g
4,Aumento relativo de masa inmunitaria,0.19438 %



**Tabla 1.** Resumen cuantitativo del escenario. El dato clave es el **porcentaje de aumento**: suele ser muy bajo, lo que demuestra que, aunque el tejido adiposo pueda albergar más células inmunes cuando crece, el sistema inmunitario global es bastante estable y no se ve “desbordado” por ese cambio local.



## 3. Conclusiones

1. La distribución inmunitaria del cuerpo es **heterogénea**: algunos tejidos pequeños son muy activos y otros grandes apenas contribuyen en densidad.
2. El tejido adiposo no es de los más densos, pero su **crecimiento** implica automáticamente más células inmunes residiendo allí.
3. El efecto **global** sobre la masa inmunitaria total es pequeño: el organismo mantiene una configuración estable incluso con más grasa.
4. El modelo usado es proporcional; en obesidad real podrían aumentar de forma preferente ciertos tipos celulares (macrófagos proinflamatorios).
5. Las visualizaciones interactivas permiten a quien corrija el trabajo explorar los datos, lo que aporta valor añadido frente a un informe estático.

## 4. Limitaciones

- Modelo basado en un único individuo de referencia (73 kg, masculino).
- Se asume densidad constante al aumentar la masa de un tejido.
- No se modelan respuestas sistémicas ni redistribución.

